In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as graph
import seaborn as sns

import cartopy.crs as ccrs

import statsmodels

import os
from tqdm import tqdm, trange

from convertbng.util import convert_bng, convert_lonlat
import utm
import re

In [2]:
def dms2dd(s):
    # example: s = """0°51'56.29"S"""
    degrees, minutes, seconds, direction = re.split('[°\'"]+', s)
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction in ('S','W'):
        dd*= -1
    return dd

In [3]:
# code and species dictionary

code_df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Alberta/To_look_at/Banff_NP_Freshwater_Lake_Fish_Index_2017_data_dictionary.csv')
# code_df = code_df.iloc[:-6]
code_df.rename(columns={'Data_Value_Valeur_de_la_donnée': 'code_name', 
                        'Value_Description_EN_Description_de_la_valeur': 'common_name'}, inplace=True)
code_df = code_df[['code_name', 'common_name']]
code_df['common_name'] = code_df['common_name'].str.lower().str.replace(' ', '_')
display(code_df.sample(5))

dict_code_name = code_df.set_index('code_name').to_dict()['common_name']

,code_name,common_name
12,OU,ouananiche
6,LKTR,lake_trout
24,BURB,burbot
9,LRSC,largescale_sucker
33,LKWH,lake_whitefish


In [4]:
new_codes = {'LNDC': 'longnose_dace', 'BNTR': 'brown_trout', 'RNTR1': 'rainbow_trout', 'CTTR1': 'cutthroat_trout', 
             'WHSC': 'white_sucker', 'BLTRCTTR(N&I)': 'bull_trout_x_cutthroat_trout', 'CTTR(hybrids)': 'cutthroat_trout', 
             'CTTRhybrids': 'cutthroat_trout', 'SUCK': 'white_sucker', 'RNWH': 'round_whitefish', 'SH': 'spottail_shiner', 
             'NSST': 'ninespince_stickleback', 'WS': 'white_sucker'}

dict_code_name.update(new_codes)

In [5]:
# Species dictionay

species_name_df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/species_name_dictionary.csv')

dict_species_name = species_name_df.set_index('common_name').to_dict()['scientific_name']

In [351]:
species_name_df = pd.DataFrame.from_dict({'common_name': list(dict_species_name.keys()), 
                                        'scientific_name' : list(dict_species_name.values())})
# species_name_df.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/species_name_dictionary.csv', index=False)

### Gros Morne

In [6]:
df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/To_look_at/GrosMorne_NP_Fish_Fence_1998-2017_data.csv')

df = df[['River', 'Date', 'atlantic_salmon', 'chinook_salmon', 'brook_trout']]
df.rename(columns=dict_species_name, inplace=True)
df['year'] = pd.DatetimeIndex(pd.to_datetime(df['Date'])).year

df['lat'], df['long'] = 'TO_GET', 'TO_GET'

df.drop(columns=['Date'])
df.replace(0, np.nan, inplace=True)
df.dropna(subset=['salmo_salar', 'oncorhynchus_tshawytscha', 'salvelinus_fontinalis'], how='all', inplace=True)
df = df[['River', 'year', 'lat', 'long', 'salmo_salar', 'oncorhynchus_tshawytscha', 'salvelinus_fontinalis']]

df.drop_duplicates(inplace=True)
df.head()

,River,year,lat,long,salmo_salar,oncorhynchus_tshawytscha,salvelinus_fontinalis
8,Trout River,2017,TO_GET,TO_GET,NaN,NaN,1.0
17,Trout River,2017,TO_GET,TO_GET,NaN,2.0,NaN
18,Trout River,2017,TO_GET,TO_GET,NaN,1.0,NaN
24,Trout River,2017,TO_GET,TO_GET,NaN,1.0,4.0
25,Trout River,2017,TO_GET,TO_GET,NaN,1.0,1.0


In [7]:
for col in ['salmo_salar', 'oncorhynchus_tshawytscha', 'salvelinus_fontinalis']:
    df[col] = np.where(df[col].notnull(), 1, df[col])
    df[col] = df[col].fillna(df.groupby(['year', 'River'])[col].transform('mean'))

df.sample(5)

,River,year,lat,long,salmo_salar,oncorhynchus_tshawytscha,salvelinus_fontinalis
674,Trout River,2011,TO_GET,TO_GET,1.0,1.0,1.0
1314,Western Brook,2006,TO_GET,TO_GET,1.0,1.0,1.0
1306,Western Brook,2006,TO_GET,TO_GET,1.0,1.0,1.0
577,Trout River,2012,TO_GET,TO_GET,1.0,1.0,1.0
1648,Western Brook,1999,TO_GET,TO_GET,1.0,1.0,1.0


In [8]:
df.rename(columns = {'River': 'waterbody_name', 'lat': 'latitude', 'long': 'longitude'}, inplace=True)
df['site_description'] = 'river'
df.head()

,waterbody_name,year,latitude,longitude,salmo_salar,oncorhynchus_tshawytscha,salvelinus_fontinalis,site_description
8,Trout River,2017,TO_GET,TO_GET,1.0,1.0,1.0,river
17,Trout River,2017,TO_GET,TO_GET,1.0,1.0,1.0,river
18,Trout River,2017,TO_GET,TO_GET,1.0,1.0,1.0,river
24,Trout River,2017,TO_GET,TO_GET,1.0,1.0,1.0,river
25,Trout River,2017,TO_GET,TO_GET,1.0,1.0,1.0,river


In [9]:
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))
display(df.head())

df.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/Processed/salmonid_fence_NO_LAT_LONG_1998_2017_occurence.csv', 
          index=False)

1205
21


,waterbody_name,year,latitude,longitude,salmo_salar,oncorhynchus_tshawytscha,salvelinus_fontinalis,site_description
8,Trout River,2017,TO_GET,TO_GET,1.0,1.0,1.0,river
85,Western Brook,2016,TO_GET,TO_GET,1.0,1.0,1.0,river
170,Deer Arm,2015,TO_GET,TO_GET,1.0,1.0,1.0,river
257,Trout River,2014,TO_GET,TO_GET,1.0,1.0,1.0,river
344,Trout River,2013,TO_GET,TO_GET,1.0,1.0,1.0,river


### Terra Nova

In [13]:
df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/TerraNova_NP_Freshwater_Pond_Angling_Stress_2005-2016_data_1.csv')
df = df[['Year', 'Date', 'Pond', 'Site Code', 'Species']]

df.replace({'A.salmon': 'atlantic_salmon', 'A_charr': 'arctic_charr', 'A_eel': 'american_eel'}, inplace=True)
df['Species'] = df['Species'].str.lower().str.replace(' ', '_')
df.replace(dict_species_name, inplace=True)

df.dropna(subset=['Species'], inplace=True)
display(df.head())

# df_sites =  pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/TerraNova_NP_Freshwater_Pond_Angling_Stress_Productivity_2007-2015_data_2.csv')
# display(df_sites.sample(5))

,Year,Date,Pond,Site Code,Species
0,2005,10-May-05,Yudle,YP22,salvelinus_fontinalis
1,2010,12-May-10,Trout,TP03,salvelinus_fontinalis
2,2005,10-May-05,Yudle,YP22,salvelinus_fontinalis
4,2010,12-May-10,Trout,TP06,salvelinus_fontinalis
6,2010,13-May-10,Trout,TP16,salvelinus_fontinalis


In [15]:
df['Species'].unique()

array(['salvelinus_fontinalis', 'salmo_salar', 'salvelinus_alpinus',
       'anguilla_rostrata', 'osmeridae_spp'], dtype=object)

In [14]:
print(df['Year'].min(), df['Year'].max())

2005 2016


### Pelagic estuary

In [17]:
df_north= pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/To_look_at/pelagic_Abondance_Nord_2008a2017.csv')
df_south = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/To_look_at/pelagic_Abondance_Sud_2008a2017.csv')

df = pd.concat([df_north, df_south])
df['year'] = pd.DatetimeIndex(pd.to_datetime(df['Date'])).year
df['lat_long'] = df['Latitude'].round(3).astype(str) + '_' + df['Longitude'].round(3).astype(str)
df.drop(columns=['Date', 'Station', 'Profondeur/Depth'], inplace=True)


dict_df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/Used/pelagic_DataDictionary.csv')
dict_df['scientific_name'] = dict_df['scientific_name'].str.lower().str.replace(' ', '_')
dict_df_species = dict_df.set_index('species').to_dict()['scientific_name']

df.rename(columns = dict_df_species, inplace=True)
df.rename(columns= dict_species_name, inplace=True)
df.columns = df.columns.str.lower()
df.replace(0, np.nan, inplace=True)
df.head()

,longitude,latitude,ammodytidae_spp,arctozenus_risso,argentina_silus,boreogadus_saida,clupea_harengus,cyclopterus_lumpus,gasterosteus_aculeatus,mallotus_villosus,melanostigma_atlanticum,merluccius_bilinearis,osmerus_mordax,pollachius_virens,scomber_scombrus,year,lat_long
0,-59.434583,47.624583,NaN,1.9,NaN,NaN,3.8,NaN,NaN,3.8,NaN,NaN,NaN,NaN,NaN,2008,47.625_-59.435
1,-60.279500,47.284917,NaN,NaN,1.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,2008,47.285_-60.28
2,-60.320500,47.795500,NaN,55.4,NaN,NaN,NaN,NaN,NaN,0.9,1.7,NaN,NaN,NaN,NaN,2008,47.796_-60.32
3,-60.559167,47.787833,NaN,18.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2008,47.788_-60.559
4,-60.647000,47.788583,NaN,1.0,1.0,NaN,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008,47.789_-60.647


In [18]:
df.columns[2:-2]

Index(['ammodytidae_spp', 'arctozenus_risso', 'argentina_silus',
       'boreogadus_saida', 'clupea_harengus', 'cyclopterus_lumpus',
       'gasterosteus_aculeatus', 'mallotus_villosus',
       'melanostigma_atlanticum', 'merluccius_bilinearis', 'osmerus_mordax',
       'pollachius_virens', 'scomber_scombrus'],
      dtype='object')

In [19]:
for col in df.columns[2:-2]:
    df[col] = np.where(df[col].notnull(), 1, df[col])
    df[col] = df[col].fillna(df.groupby(['year', 'lat_long'])[col].transform('mean'))

df.sample(5)

,longitude,latitude,ammodytidae_spp,arctozenus_risso,argentina_silus,boreogadus_saida,clupea_harengus,cyclopterus_lumpus,gasterosteus_aculeatus,mallotus_villosus,melanostigma_atlanticum,merluccius_bilinearis,osmerus_mordax,pollachius_virens,scomber_scombrus,year,lat_long
422,-64.217000,47.145667,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2010,47.146_-64.217
424,-64.389833,47.759333,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,2010,47.759_-64.39
770,-63.342500,46.854333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2012,46.854_-63.342
1121,-60.413750,47.591500,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,2014,47.592_-60.414
1451,-60.493333,47.486500,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,47.486_-60.493


In [20]:
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))

display(df.sample(5))
df.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/Processed/pelagic_abondance_2008_2017_occurence.csv', 
          index=False)

3281
3281


,longitude,latitude,ammodytidae_spp,arctozenus_risso,argentina_silus,boreogadus_saida,clupea_harengus,cyclopterus_lumpus,gasterosteus_aculeatus,mallotus_villosus,melanostigma_atlanticum,merluccius_bilinearis,osmerus_mordax,pollachius_virens,scomber_scombrus,year,lat_long
666,-65.706917,49.442750,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,2011,49.443_-65.707
1077,-61.114000,46.979500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2014,46.98_-61.114
700,-60.194917,48.872833,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2012,48.873_-60.195
553,-63.989500,48.993833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2011,48.994_-63.99
365,-61.784333,46.779833,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2010,46.78_-61.784


### NE Newfoundland Cod

In [23]:
df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/Used/NE_newfounland_cod.csv')
df = df[['year', 'decimalLatitude', 'decimalLongitude', 'vernacularName']]
df.rename(columns={'decimalLatitude': 'lat', 'decimalLongitude': 'long'}, inplace=True)

df['lat_long'] = df['lat'].round(3).astype(str) + '_' + df['long'].round(3).astype(str)
df['gadus_morhua'] = 1

df.drop(columns=['vernacularName'], inplace=True)
df.drop_duplicates(inplace=True)

print(f"Years: {df['year'].min(), df['year'].max()}")
display(df.sample(5))
df.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/Processed/NE_newfounland_cod_2005_2011_occurence.csv', 
          index=False)

Years: (2005, 2011)


,year,lat,long,lat_long,gadus_morhua
714,2006,48.08508,-53.85343,48.085_-53.853,1
377,2011,48.58000,-53.88980,48.58_-53.89,1
787,2006,47.72092,-52.80357,47.721_-52.804,1
342,2009,48.70110,-53.15080,48.701_-53.151,1
178,2008,48.15150,-53.73080,48.152_-53.731,1


### Demersal Abondance

In [24]:
df_north= pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/To_look_at/demersal_Abondance_Nord_2008a2017.csv')
df_south = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/To_look_at/demersal_Abondance_Sud_2008a2017.csv')

df = pd.concat([df_north, df_south])
df['year'] = pd.DatetimeIndex(pd.to_datetime(df['Date'])).year
df['lat_long'] = df['Latitude'].round(3).astype(str) + '_' + df['Longitude'].round(3).astype(str)
df.drop(columns=['Date', 'Station', 'Profondeur/Depth'], inplace=True)


dict_df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/Used/demersal_DataDictionary.csv')
dict_df['scientific_name'] = dict_df['scientific_name'].str.lower().str.replace(' ', '_')
dict_df_species = dict_df.set_index('species').to_dict()['scientific_name']

df.rename(columns = dict_df_species, inplace=True)
df.rename(columns = dict_species_name, inplace=True)
df.columns = df.columns.str.lower()
df.replace(0, np.nan, inplace=True)
print(df['year'].min(), df['year'].max())
df.head()

2008 2017


,longitude,latitude,alosa_pseudoharengus,amblyraja_radiata,anarhichas_lupus,anisarchus_medius,anarhichas_minor,artediellus_atlanticus,artediellus_uncinatus,aspidophoroides_monopterygius,...,sebastes_spp,squalus_acanthias,stichaeus_punctatus,tautogolabrus_adspersus,triglops_murrayi,aspidophoroides_olrikii,urophycis_tenuis,zoarces_americanus,year,lat_long
0,-59.434583,47.624583,NaN,6.6,NaN,NaN,NaN,NaN,NaN,NaN,...,158.4,NaN,NaN,NaN,NaN,NaN,15.9,NaN,2008,47.625_-59.435
1,-60.279500,47.284917,NaN,NaN,1.0,NaN,NaN,5.0,NaN,NaN,...,1388.0,NaN,NaN,NaN,NaN,NaN,27.0,NaN,2008,47.285_-60.28
2,-60.320500,47.795500,NaN,1.7,NaN,NaN,NaN,NaN,NaN,NaN,...,37.5,NaN,NaN,NaN,NaN,NaN,0.9,NaN,2008,47.796_-60.32
3,-60.559167,47.787833,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,...,280.0,NaN,NaN,NaN,NaN,NaN,14.0,NaN,2008,47.788_-60.559
4,-60.647000,47.788583,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2141.0,NaN,NaN,NaN,NaN,NaN,51.0,NaN,2008,47.789_-60.647


In [25]:
for col in df.columns[2:-2]:
    df[col] = np.where(df[col].notnull(), 1, df[col])
    df[col] = df[col].fillna(df.groupby(['year', 'lat_long'])[col].transform('mean'))

df.sample(5)

,longitude,latitude,alosa_pseudoharengus,amblyraja_radiata,anarhichas_lupus,anisarchus_medius,anarhichas_minor,artediellus_atlanticus,artediellus_uncinatus,aspidophoroides_monopterygius,...,sebastes_spp,squalus_acanthias,stichaeus_punctatus,tautogolabrus_adspersus,triglops_murrayi,aspidophoroides_olrikii,urophycis_tenuis,zoarces_americanus,year,lat_long
1480,-63.590667,49.058750,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2016,49.059_-63.591
1432,-61.510000,46.371667,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2017,46.372_-61.51
45,-62.744000,47.033333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,2008,47.033_-62.744
1508,-64.410333,47.808500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,47.808_-64.41
977,-62.644500,45.944667,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,2014,45.945_-62.644


In [26]:
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))

display(df.sample(5))
df.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/Processed/demersal_abondance_2008_2017_occurence.csv', 
          index=False)

3281
3281


,longitude,latitude,alosa_pseudoharengus,amblyraja_radiata,anarhichas_lupus,anisarchus_medius,anarhichas_minor,artediellus_atlanticus,artediellus_uncinatus,aspidophoroides_monopterygius,...,sebastes_spp,squalus_acanthias,stichaeus_punctatus,tautogolabrus_adspersus,triglops_murrayi,aspidophoroides_olrikii,urophycis_tenuis,zoarces_americanus,year,lat_long
172,-67.822917,48.811167,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2008,48.811_-67.823
345,-65.482833,49.359833,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009,49.36_-65.483
490,-61.261500,46.461500,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2010,46.462_-61.262
1511,-64.339167,47.321167,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,47.321_-64.339
834,-63.016333,46.526333,NaN,NaN,1.0,NaN,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,2013,46.526_-63.016


### BEach Seine

In [27]:
df_dict = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/Used/beach_seine_dict.csv')
df_dict['full_scientific_name'] = df_dict['col_name'].str.split('[').str.get(0).str.rstrip().str.lower().str.replace(' ', '_')
df_dict['col_name'] = df_dict['col_name'].str.split('(').str.get(1).str.split(')').str.get(0) + ' [#]'
display(df_dict.head())

df_dict_names = df_dict.set_index('col_name').to_dict()['full_scientific_name']

,col_name,full_scientific_name
0,H. platessoides [#],hippoglossoides_platessoides
1,L. piscatorius [#],lophius_piscatorius
2,S. punctatus punctatus [#],stichaeus_punctatus_punctatus
3,G. morhua [#],gadus_morhua
4,G. morhua [#],gadus_morhua


In [28]:
df= pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/To_look_at/beach_seine_data.csv')

df['lat_long'] = df['lat'].round(3).astype(str) + '_' + df['long'].round(3).astype(str)
df.rename(columns=df_dict_names, inplace=True)
df.rename(columns = {'ammodytes_spp.': 'ammodytes', 'lat': 'latitude', 'long': 'longitude'}, inplace=True)
df.replace(0, np.nan, inplace=True)
df.rename(columns=dict_species_name, inplace=True)
df.head()

,latitude,longitude,year,hippoglossoides_platessoides,lophius_piscatorius,stichaeus_punctatus,gadus_morhua,clupea_harengus,scomber_scombrus,salmo_salar,...,rajidae_indeterminata,lumpenus_lampretaeformis,raja_montagui,gasterosteus_aculeatus,anisarchus_medius,amblyraja_radiata,urophycis_tenuis,pseudopleuronectes_americanus,limanda_ferruginea,lat_long
0,47.9823,-53.6478,1960,NaN,NaN,NaN,215.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,47.982_-53.648
1,47.5733,-53.2617,1960,NaN,NaN,NaN,486.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.573_-53.262
2,47.5733,-53.2600,1960,NaN,NaN,NaN,1282.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,1.0,NaN,47.573_-53.26
3,47.7433,-53.1983,1960,NaN,NaN,NaN,100.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,43.0,1.0,NaN,47.743_-53.198
4,47.6750,-53.1883,1960,NaN,NaN,NaN,235.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,47.675_-53.188


In [29]:
sorted(df.columns[3:-1])

['amblyraja_radiata',
 'ammodytidae_spp',
 'anguilla_rostrata',
 'anisarchus_medius',
 'clupea_harengus',
 'cottidae_spp',
 'cyclopterus_lumpus',
 'dactylopterus_volitans',
 'gadus_morhua',
 'gadus_ogac',
 'gasterosteus_aculeatus',
 'hemitripterus_americanus',
 'hippoglossoides_platessoides',
 'leptoclinus_maculatus',
 'limanda_ferruginea',
 'lophius_piscatorius',
 'lumpenus_lampretaeformis',
 'mallotus_villosus',
 'melanogrammus_aeglefinus',
 'osmerus_mordax',
 'peprilus_triacanthus',
 'pholis_fasciata',
 'pholis_gunnellus',
 'pseudopleuronectes_americanus',
 'raja_montagui',
 'rajidae_indeterminata',
 'salmo_salar',
 'salvelinus_fontinalis',
 'scomber_scombrus',
 'stichaeus_punctatus',
 'tautogolabrus_adspersus',
 'ulvaria_subbifurcata',
 'urophycis_tenuis',
 'zoarces_americanus']

In [30]:
for col in df.columns[3:-1]:
    df[col] = np.where(df[col].notnull(), 1, df[col])
    df[col] = df[col].fillna(df.groupby(['year', 'lat_long'])[col].transform('mean'))

df.sample(5)

,latitude,longitude,year,hippoglossoides_platessoides,lophius_piscatorius,stichaeus_punctatus,gadus_morhua,clupea_harengus,scomber_scombrus,salmo_salar,...,rajidae_indeterminata,lumpenus_lampretaeformis,raja_montagui,gasterosteus_aculeatus,anisarchus_medius,amblyraja_radiata,urophycis_tenuis,pseudopleuronectes_americanus,limanda_ferruginea,lat_long
134,46.7517,-53.3650,1992,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.752_-53.365
228,48.3917,-53.3717,1994,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,48.392_-53.372
186,48.1517,-53.7300,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,48.152_-53.73
219,47.7433,-53.1983,1994,NaN,NaN,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,47.743_-53.198
139,47.7217,-53.1950,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.722_-53.195


In [31]:
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))

display(df.sample(5))
df.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/Processed/beach_seine_1960_1996_occurence.csv', 
          index=False)

325
325


,latitude,longitude,year,hippoglossoides_platessoides,lophius_piscatorius,stichaeus_punctatus,gadus_morhua,clupea_harengus,scomber_scombrus,salmo_salar,...,rajidae_indeterminata,lumpenus_lampretaeformis,raja_montagui,gasterosteus_aculeatus,anisarchus_medius,amblyraja_radiata,urophycis_tenuis,pseudopleuronectes_americanus,limanda_ferruginea,lat_long
76,47.6750,-53.1883,1963,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,47.675_-53.188
49,47.8400,-53.8733,1962,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.84_-53.873
208,49.4917,-56.0650,1993,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,49.492_-56.065
227,48.1643,-53.7109,1994,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,48.164_-53.711
223,47.8400,-53.8733,1994,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.84_-53.873


### AFSC

In [34]:
df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/Used/AFSC_ABL_Stock_composition_Chinook_salmon.csv')
df = df[['YEAR', 'lat', 'long']]

df['oncorhynchus_tshawytscha'] = 1
df.dropna(inplace=True)
df['lat_long'] = df['lat'].round(3).astype(str) + '_' + df['long'].round(3).astype(str)
df.drop_duplicates(inplace=True)
df.rename(columns = {'lat': 'latitude', 'long': 'longitude', 'YEAR': 'year'}, inplace=True)

print(len(df), df['year'].min(), df['year'].max())

display(df.head())
df.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/Used/AFSC_2002_2004.csv', index=False)

2736 2002 2004


,year,latitude,longitude,oncorhynchus_tshawytscha,lat_long
0,2002,61.7675,-162.0226,1,61.768_-162.023
1,2002,61.7534,-162.0101,1,61.753_-162.01
2,2002,61.7619,-162.0226,1,61.762_-162.023
3,2002,61.7634,-162.0175,1,61.763_-162.018
4,2002,61.7698,-162.0222,1,61.77_-162.022


### Nuseeds salmon

In [40]:
df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/To_look_at/nuseds_salmon.csv')
df.replace({'CK': 'chinook_salmon', 'CO': 'coho_salmon', 'PKE': 'pink_salmon', 'PKO': 'pink_salmon', 
            'SEL': 'sockeye_salmon', 'SER': 'sockeye_salmon', 'CM': 'chum_salmon'}, inplace=True)
df.replace(dict_species_name, inplace=True)

df.rename(columns={'CU_LAT': 'lat', 'CU_LONGT': 'long'}, inplace=True)
df = df[['SPECIES_QUALIFIED', 'lat', 'long', 'EFFECTIVE_DT', 'SYSTEM_SITE', 'GFE_TYPE']]

df['lat_long'] = df['lat'].round(3).astype(str) + '_' + df['long'].round(3).astype(str)
df['year'] = pd.DatetimeIndex(pd.to_datetime(df['EFFECTIVE_DT'])).year
df.drop(columns=['EFFECTIVE_DT'], inplace=True)

df.rename(columns = {'lat': 'latitude', 'long': 'longitude', 'SYSTEM_SITE': 'waterbody_name', 
                     'GFE_TYPE': 'site_description'}, inplace=True)
df.head()

,SPECIES_QUALIFIED,latitude,longitude,waterbody_name,site_description,lat_long,year
0,oncorhynchus_tshawytscha,49.076370,-119.518180,OKANAGAN RIVER,Stream,49.076_-119.518,2011
1,oncorhynchus_kisutch,49.063247,-122.786144,CAMPBELL RIVER,Stream,49.063_-122.786,2011
2,oncorhynchus_kisutch,49.063247,-122.786144,MURRAY CREEK,Stream,49.063_-122.786,2011
3,oncorhynchus_kisutch,49.063247,-122.786144,NICOMEKL RIVER,Stream,49.063_-122.786,2011
4,oncorhynchus_kisutch,49.063247,-122.786144,SERPENTINE RIVER,Stream,49.063_-122.786,2011


In [41]:
for col in tqdm(df['SPECIES_QUALIFIED'].unique()):
    df[col] = np.where(df.isin([col]).any(1), 1, np.nan)
    df[col] = df[col].fillna(df.groupby(['year', 'lat_long'])[col].transform('mean'))

display(df.head())

100%|██████████| 5/5 [00:00<00:00, 86.19it/s]


,SPECIES_QUALIFIED,latitude,longitude,waterbody_name,site_description,lat_long,year,oncorhynchus_tshawytscha,oncorhynchus_kisutch,oncorhynchus_gorbuscha,oncorhynchus_keta,oncorhynchus_nerka
0,oncorhynchus_tshawytscha,49.076370,-119.518180,OKANAGAN RIVER,Stream,49.076_-119.518,2011,1.0,NaN,NaN,NaN,1.0
1,oncorhynchus_kisutch,49.063247,-122.786144,CAMPBELL RIVER,Stream,49.063_-122.786,2011,NaN,1.0,NaN,NaN,NaN
2,oncorhynchus_kisutch,49.063247,-122.786144,MURRAY CREEK,Stream,49.063_-122.786,2011,NaN,1.0,NaN,NaN,NaN
3,oncorhynchus_kisutch,49.063247,-122.786144,NICOMEKL RIVER,Stream,49.063_-122.786,2011,NaN,1.0,NaN,NaN,NaN
4,oncorhynchus_kisutch,49.063247,-122.786144,SERPENTINE RIVER,Stream,49.063_-122.786,2011,NaN,1.0,NaN,NaN,NaN


In [42]:
print(df['year'].min(), df['year'].max())

2011 2020


In [43]:
df.drop(columns=['SPECIES_QUALIFIED'], inplace=True)
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))

display(df.sample(5))
df.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/Processed/nuseds_salmon_2011_2020_occurence.csv', 
          index=False)

7145
7061


,latitude,longitude,waterbody_name,site_description,lat_long,year,oncorhynchus_tshawytscha,oncorhynchus_kisutch,oncorhynchus_gorbuscha,oncorhynchus_keta,oncorhynchus_nerka
1171,50.841085,-119.750360,LITTLE RIVER,Stream Segment,50.841_-119.75,2011,1.0,NaN,NaN,NaN,NaN
207,49.428659,-126.024918,BINGO CREEK,Stream,49.429_-126.025,2011,NaN,NaN,NaN,1.0,NaN
311,49.428659,-126.024918,MCKAY COVE CREEK,Stream,49.429_-126.025,2011,NaN,NaN,NaN,1.0,NaN
7049,52.642426,-128.711697,CHAMISS CREEK,Stream,52.642_-128.712,2011,NaN,NaN,NaN,NaN,1.0
5011,55.067697,-126.698870,PINKUT CREEK,Stream,55.068_-126.699,2011,1.0,NaN,NaN,NaN,NaN


### JEanette Salmon

In [47]:
df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/To_look_at/jeanette_salmon.csv')
df = df[['species', 'year', 'lat', 'long', 'LocationID']]

df.replace({'anadromous dolly varden': 'dolly_varden_trout', 'chinook': 'chinook_salmon', 'chum': 'chum_salmon', 
           'coho': 'coho_salmon', 'general cutthroat trout': 'cutthroat_trout', 'general dolly varden': 'dolly_varden_trout', 
           'pink': 'pink_salmon', 'sheefish (inconnu)': 'sheefish', 'sockeye': 'sockeye_salmon', 
           'steelhead down': 'steelhead_trout', 'steelhead trout': 'steelhead_trout', 'steelhead up': 'steelhead_trout', 
           'unknown salmon': 'salmonidae', 'Chinook': 'chinook_salmon', 'cutthroat trout': 'cutthroat_trout', 
           'whitefish, broad': 'broad_whitefish', 'whitefish, humpback': 'humpback_whitefish', 
           'whitefish, least cisco': 'least_cisco', 'steelhead': 'steelhead_trout', 'arctic char': 'arctic_char', 
           'dolly varden (anadromous)': 'dolly_varden_trout', 'dolly varden (general)': 'dolly_varden_trout', 
            'dolly varden': 'dolly_varden_trout'}, inplace=True)
df = df[~df['species'].isin(['mixed', 'other (non-target)'])]

df['lat_long'] = df['lat'].round(3).astype(str) + '_' + df['long'].round(3).astype(str)
df.replace(dict_species_name, inplace=True)
df.rename(columns = {'lat': 'latitude', 'long': 'longitude', 'LocationID': 'waterbody_name'}, inplace=True)

df.head()

,species,year,latitude,longitude,waterbody_name,lat_long
0,salvelinus_malma,1962,57.40284,-135.1033,Lake Eva,57.403_-135.103
1,oncorhynchus_tshawytscha,1987,56.03890,-160.2734,Bear River,56.039_-160.273
2,oncorhynchus_tshawytscha,1989,56.03890,-160.2734,Bear River,56.039_-160.273
3,oncorhynchus_tshawytscha,1990,56.03890,-160.2734,Bear River,56.039_-160.273
4,oncorhynchus_tshawytscha,1991,56.03890,-160.2734,Bear River,56.039_-160.273


In [48]:
for col in df['species'].unique():
    df[col] = np.where(df.isin([col]).any(1), 1, np.nan)
    df[col] = df[col].fillna(df.groupby(['year', 'lat_long'])[col].transform('mean'))

display(df.head())

,species,year,latitude,longitude,waterbody_name,lat_long,salvelinus_malma,oncorhynchus_tshawytscha,oncorhynchus_keta,oncorhynchus_kisutch,oncorhynchus_clarkii,oncorhynchus_gorbuscha,stenodus_leucichthys,oncorhynchus_nerka,oncorhynchus_mykiss,salmonidae_spp,coregonus_nasus,coregonus_clupeaformis,coregonus_sardinella,salvelinus_alpinus
0,salvelinus_malma,1962,57.40284,-135.1033,Lake Eva,57.403_-135.103,1.0,NaN,1.0,1.0,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,oncorhynchus_tshawytscha,1987,56.03890,-160.2734,Bear River,56.039_-160.273,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,oncorhynchus_tshawytscha,1989,56.03890,-160.2734,Bear River,56.039_-160.273,NaN,1.0,1.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,oncorhynchus_tshawytscha,1990,56.03890,-160.2734,Bear River,56.039_-160.273,NaN,1.0,1.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,oncorhynchus_tshawytscha,1991,56.03890,-160.2734,Bear River,56.039_-160.273,NaN,1.0,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
df['year'].max()

2017

In [50]:
df.drop(columns=['species'], inplace=True)
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))

display(df.sample(5))
df.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/Processed/jeanette_salmon_1921_2017_occurence.csv', 
          index=False)

17951
6484


,year,latitude,longitude,waterbody_name,lat_long,salvelinus_malma,oncorhynchus_tshawytscha,oncorhynchus_keta,oncorhynchus_kisutch,oncorhynchus_clarkii,oncorhynchus_gorbuscha,stenodus_leucichthys,oncorhynchus_nerka,oncorhynchus_mykiss,salmonidae_spp,coregonus_nasus,coregonus_clupeaformis,coregonus_sardinella,salvelinus_alpinus
873,1993,58.77501,-132.0797,Nahlin River,58.775_-132.08,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
11377,2004,61.50958,-159.3819,Aniak River,61.51_-159.382,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
1878,2006,55.77408,-133.4639,Chuck Lake,55.774_-133.464,1.0,NaN,1.0,1.0,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
646,2010,61.91917,-156.2431,Tatlawiksuk River,61.919_-156.243,NaN,1.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
9446,2011,NaN,NaN,South Peninsula,nan_nan,NaN,1.0,1.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


### Rohonczy

In [51]:
df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/Used/rohonczy.csv')
df['lat'] = df['lat'].astype(str) + 'N'
df['long'] = df['long'].astype(str) + 'W'

df['lat'] = df['lat'].apply(dms2dd)
df['long'] = df['long'].apply(dms2dd)
df['lat_long'] = df['lat'].round(3).astype(str) + '_' + df['long'].round(3).astype(str)

df.drop(columns=['Collection Date', 'Field Site ID'], inplace=True)
df.drop_duplicates(inplace=True)

df = df[~df['Species'].isin(['INNC'])]

df.replace(dict_code_name, inplace=True)
df.replace(dict_species_name, inplace=True)

print(df['year'].min(), df['year'].max())
display(df.head())

2013 2015


,Species,year,lat,long,lat_long
0,coregonus_clupeaformis,2015,62.276667,-114.193056,62.277_-114.193
12,coregonus_artedi,2015,62.276667,-114.193056,62.277_-114.193
15,prosopium_cylindraceum,2015,62.276667,-114.193056,62.277_-114.193
19,salvelinus_namaycush,2015,62.276667,-114.193056,62.277_-114.193
20,catostomus_catostomus,2015,62.276667,-114.193056,62.277_-114.193


In [52]:
for col in df['Species'].unique():
    df[col] = np.where(df.isin([col]).any(1), 1, np.nan)
    df[col] = df[col].fillna(df.groupby(['year', 'lat_long'])[col].transform('mean'))

display(df.head())

,Species,year,lat,long,lat_long,coregonus_clupeaformis,coregonus_artedi,prosopium_cylindraceum,salvelinus_namaycush,catostomus_catostomus,lota_lota,esox_lucius,pungitius_pungitius,sander_vitreus,cottus_cognatus,notropis_hudsonius,catostomus_commersonii
0,coregonus_clupeaformis,2015,62.276667,-114.193056,62.277_-114.193,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
12,coregonus_artedi,2015,62.276667,-114.193056,62.277_-114.193,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
15,prosopium_cylindraceum,2015,62.276667,-114.193056,62.277_-114.193,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
19,salvelinus_namaycush,2015,62.276667,-114.193056,62.277_-114.193,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
20,catostomus_catostomus,2015,62.276667,-114.193056,62.277_-114.193,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN


In [53]:
df.drop(columns=['Species'], inplace=True)
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))

display(df.sample(5))
df.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/Processed/rohonczy_2013_2015_occurence.csv', 
          index=False)

68
30


,year,lat,long,lat_long,coregonus_clupeaformis,coregonus_artedi,prosopium_cylindraceum,salvelinus_namaycush,catostomus_catostomus,lota_lota,esox_lucius,pungitius_pungitius,sander_vitreus,cottus_cognatus,notropis_hudsonius,catostomus_commersonii
167,2015,62.376944,-114.247778,62.377_-114.248,1.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,1.0,1.0,NaN
103,2014,62.487917,-114.359194,62.488_-114.359,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN
90,2013,62.398361,-114.329222,62.398_-114.329,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
127,2014,62.487750,-114.327278,62.488_-114.327,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
98,2014,62.486639,-114.360333,62.487_-114.36,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN


### ASL

In [61]:
df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/To_look_at/ASL_merged.csv')

df = df[['sampleDate', 'Location', 'Species']]
df.head()

,sampleDate,Location,Species
0,1967-01-01,Andreafsky River (East Fork),Chum
1,1967-01-01,Andreafsky River (East Fork),Chum
2,1967-01-01,Andreafsky River (East Fork),Chum
3,1967-01-01,Andreafsky River (East Fork),Chum
4,1967-01-01,Andreafsky River (East Fork),Chum


In [62]:
df.replace({'Chum': 'chum_salmon', 'Chinook': 'chinook_salmon', 'Sockeye': 'sockeye_salmon', 'Coho': 'coho_salmon', 
            'Pink': 'pink_salmon'}, inplace=True)
df['Species'].replace(dict_species_name, inplace=True)
df['year'] = pd.DatetimeIndex(pd.to_datetime(df['sampleDate'])).year

df['lat'], df['long'] = 'TO_GET', 'TO_GET'

print(len(df))
df.drop(columns=['sampleDate'], inplace=True)
df.drop_duplicates(inplace=True)
print(len(df))

display(df.sample(5))

1766224
4245


,Location,Species,year,lat,long
248688,Goodnews River (Middle Fork),oncorhynchus_tshawytscha,1996,TO_GET,TO_GET
670434,Moses Point (Village/City),oncorhynchus_keta,1990,TO_GET,TO_GET
147922,Dogfish Village (Village/City),oncorhynchus_keta,1968,TO_GET,TO_GET
723909,Norton Bay Subdistrict,oncorhynchus_keta,2009,TO_GET,TO_GET
1209492,W5 (Goodnews Bay Subdistrict),oncorhynchus_tshawytscha,2002,TO_GET,TO_GET


In [63]:
for col in df['Species'].unique():
    df[col] = np.where(df.isin([col]).any(1), 1, np.nan)
    df[col] = df[col].fillna(df.groupby(['year', 'Location'])[col].transform('mean'))

display(df.head())

,Location,Species,year,lat,long,oncorhynchus_keta,oncorhynchus_tshawytscha,oncorhynchus_nerka,oncorhynchus_kisutch,oncorhynchus_gorbuscha
0,Andreafsky River (East Fork),oncorhynchus_keta,1967,TO_GET,TO_GET,1.0,NaN,NaN,NaN,NaN
113,Andreafsky River (East Fork),oncorhynchus_tshawytscha,1980,TO_GET,TO_GET,NaN,1.0,NaN,NaN,NaN
148,Andreafsky River (East Fork),oncorhynchus_tshawytscha,1981,TO_GET,TO_GET,1.0,1.0,NaN,NaN,NaN
168,Andreafsky River (East Fork),oncorhynchus_keta,1981,TO_GET,TO_GET,1.0,1.0,NaN,NaN,NaN
716,Andreafsky River (East Fork),oncorhynchus_tshawytscha,1982,TO_GET,TO_GET,1.0,1.0,NaN,NaN,NaN


In [64]:
df.rename(columns = {'Location': 'waterbody_name', 'lat': 'latitude', 'long': 'longitude'}, inplace=True)

In [65]:
print(df['year'].min(), df['year'].max())

1960 2016


In [66]:
df.drop(columns=['Species'], inplace=True)
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))

display(df.sample(5))
df.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/Processed/asl_salmon_NO_LAT_LONG_1960_2016_occurence.csv', 
          index=False)

4245
2280


,waterbody_name,year,latitude,longitude,oncorhynchus_keta,oncorhynchus_tshawytscha,oncorhynchus_nerka,oncorhynchus_kisutch,oncorhynchus_gorbuscha
866916,Takotna River,2002,TO_GET,TO_GET,1.0,1.0,NaN,1.0,NaN
1051423,Johnson River,1974,TO_GET,TO_GET,NaN,1.0,NaN,NaN,NaN
1707560,Bethel - subdistrict W1A (Above Bethel),2011,TO_GET,TO_GET,1.0,1.0,1.0,1.0,NaN
614926,Big Eddy,1985,TO_GET,TO_GET,1.0,1.0,NaN,1.0,NaN
525316,Kwethluk River,2010,TO_GET,TO_GET,1.0,1.0,1.0,1.0,NaN


### Yukon and NWT Presence Data

In [67]:
df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/Used/Yukon_NWT_PDF_Conversions.csv')

df['year'] = df['Date'].str[-4:].astype(int)
df = df[['Latitude', 'Longitude', 'year', 'waterbody', 'Water Type', 'Species']]

df['Species'] = df['Species'].str.lower().str.replace(' ', '_')
df = df[~df['Species'].isin(['inconnu'])]

df.replace(dict_species_name, inplace=True)
print(df['year'].min(), df['year'].max())
display(df.sample(5))

1980 2019


,Latitude,Longitude,year,waterbody,Water Type,Species
179,TO_GET,TO_GET,2009,Pine Lake,Lake,salvelinus_namaycush
82,60.701695,-135.278336,2011,Louise (Jackson) Lake,Lake,salvelinus_namaycush
71,64.010697,-135.258626,2017,Ladue Lake,Lake,coregonus_clupeaformis
48,60.08854,-134.857843,2014,Bennet Lake,Lake,salvelinus_namaycush
183,TO_GET,TO_GET,2002,Pine Lake,Lake,esox_lucius


In [68]:
for col in df['Species'].unique():
    df[col] = np.where(df.isin([col]).any(1), 1, np.nan)
    df[col] = df[col].fillna(df.groupby(['year', 'waterbody'])[col].transform('mean'))

display(df.sample(5))

,Latitude,Longitude,year,waterbody,Water Type,Species,salvelinus_namaycush,thymallus_arcticus,coregonus_nasus,lota_lota,...,esox_lucius,prosopium_cylindraceum,cottus_cognatus,oncorhynchus_tshawytscha,oncorhynchus_keta,oncorhynchus_mykiss,salvelinus_alpinus,oncorhynchus_nerka,salvelinus_confluentus,coregonus_autumnalis
161,TO_GET,TO_GET,1994,Tagish River,River,salvelinus_namaycush,1.0,1.0,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,TO_GET,TO_GET,1997,Teslin Lake,Lake,thymallus_arcticus,1.0,1.0,1.0,1.0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,TO_GET,TO_GET,1998,Lubbock River,River,thymallus_arcticus,NaN,1.0,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
298,TO_GET,TO_GET,2013,Dezadeash Lake,Lake,salvelinus_namaycush,1.0,1.0,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,TO_GET,TO_GET,2000,Teslin Lake,Lake,thymallus_arcticus,1.0,1.0,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
df.rename(columns = {'Latitude': 'latitude', 'Longitude': 'longitude', 'waterbody': 'waterbody_name', 
                     'Water Type': 'site_description'}, inplace=True)

In [70]:
df.drop(columns=['Species'], inplace=True)
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))

display(df.sample(5))
df.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Victor_files/Processed/yukon_NWT_1980_2019_occurence.csv', 
          index=False)

334
120


,latitude,longitude,year,waterbody_name,site_description,salvelinus_namaycush,thymallus_arcticus,coregonus_nasus,lota_lota,coregonus_clupeaformis,...,esox_lucius,prosopium_cylindraceum,cottus_cognatus,oncorhynchus_tshawytscha,oncorhynchus_keta,oncorhynchus_mykiss,salvelinus_alpinus,oncorhynchus_nerka,salvelinus_confluentus,coregonus_autumnalis
1,60.155535,-132.742918,2003,Teslin Lake,Lake,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,62.615296,-135.949362,2011,Ta'tla Mun,Lake,1.0,NaN,NaN,1.0,1.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,60.377889,-134.831388,2010,Lewes Lake,Lake,1.0,1.0,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
326,TO_GET,TO_GET,1993,Raddi Lake,Lake,1.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
91,63.686079,-136.163072,2014,Minto Lake,Lake,1.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
